# Microestructuras y Sistemas de Trading
## Laboratorio 1
### Equipo C

<img style="float: right; margin: -100px 80px 30px 0px;" src="https://upload.wikimedia.org/wikipedia/commons/d/db/Logo_ITESO_normal.jpg" width="200px" height="200px" />

#### <font color="blue">**INTEGRANTES:**</font>

* <font color="black">**INTEGRANTE 1 :**</font> Andres García Ruvalcaba
* <font color="black">**INTEGRANTE 2 :**</font> Mariana López Correa,
* <font color="black">**INTEGRANTE 3 :**</font> Paulina Milenka Glez Manrique 
* <font color="black">**INTEGRANTE 4 :**</font> Juan López Pelayo

<font color="red">**Repositorio :**</font> https://github.com/paulinamilenka/LAB-1

##  <span style="color:blue">**1.1 Objetivos**</span>

#### 1.2.1 Objetivo general.

> Replicar los ETFs conocidos como ishares para el mercado de renta varialbe de USA que la empresa BlackRock ofrece para analizar la construcción de un portafolio de inversión y un criterio de rebalanceo. 

#### 1.2.2 Objetivo específico.

> * Comparar los resultados de haber rebalanceado periódicamente el portafolio o no haberlo hecho y continuar todo un año con     las mismas posiciones del ETF IVW.
> * Realizar el proceso para que pueda replicarse para cualquier ETF del mercado. 

## Introducción 

### <font color = "green" > **¿Qué es un ETF?**:
    
> Un ETF (Exchange Trade Fund)  puede definirse como un fondo que replica un índice, es decir, es un fondo que cotiza en bolsa y   que está compuesto por un grupo de activos diversificados, pues contiene las acciones de todas las empresas de un índice.

## Desarrollo

In [14]:
import quandl 
import pandas as pd
import os
import calendar
import numpy as np

In [15]:
quandl.ApiConfig.api_key = "6ARMqMT_1jbzJ71XyRCx"

In [16]:
path=os.getcwd()
files=os.listdir(path)
etf_files = [f for f in files if f[-3:]=='csv']

In [17]:
df=[]
for f in etf_files: 
    data=pd.read_csv(f, header=0, sep=',', index_col=0, parse_dates=False, skip_blank_lines=True, skiprows=10)

    df.append(data)

In [18]:
df_nassets = []
for f in etf_files:
    data=pd.read_csv(f,nrows=4)
    data=data.iloc[3,0]
    df_nassets.append(data)

In [19]:
nassets = []
for i in range(len(df_nassets)):
    a=df_nassets[i]
    a=a.replace('.00','').replace(',','')
    a=int(a)
    nassets.append(a)

In [20]:
df_fechas=[]
for f in etf_files:
    data=pd.read_csv(f,nrows=4)
    data=data.iloc[1,0]
    df_fechas.append(data)

In [21]:
dic = {'abr':'04','ago':'08','dic':'12','feb':'02','ene':'01','jun':'06','jul':'07','mar':'03','may':'05','nov':'11','oct':'10','sep':'09'}
lenfechas=len(df_fechas)
for i in range(lenfechas):
    df_fechas[i]=df_fechas[i].replace(df_fechas[i][3:6],dic.get(df_fechas[i][3:6]))

In [22]:
l=len(df_fechas)
for i in range(l):
    df_fechas[i]=df_fechas[i][-4:]+df_fechas[i][2:6]+df_fechas[i][0:2]


In [23]:
df_prices = []
for i in range(len(df)-1):
    arch= df[i]
    tickers = arch.index.get_values()
    tickers = tickers.tolist()
    prices = quandl.get_table('WIKI/PRICES',ticker = tickers,  qopts = { 'columns': ['ticker', 'date', 'adj_close'] }, 
                       date = { 'gte': df_fechas[i], 'lte': df_fechas[-1] }, 
                       paginate=True)
    df_prices.append(prices)

In [24]:
p_prices=df_prices[0].pivot('ticker','date','adj_close')
#p_prices=p_prices.fillna(p_prices.mean(axis=1))
p_prices=p_prices.fillna(method='ffill',axis=1)

In [26]:
q_acc=[]
l=len(df)
for i in range(l):
    data=df[i]['Acciones'].dropna()
    data=data.sort_index(ascending=True)
    q_acc.append(data)

In [27]:
acciones=[] #lista con las acciones cinvertida a DataFrame
l=len(q_acc)
for i in range(l):
    data=q_acc[i].to_frame()
    acciones.append(data)


In [28]:
l=len(acciones)
acc_clean = [] #Lista con las acciones limpias 
for i in range(l):
    a=acciones[i]
    a=a['Acciones'].str.replace('.00','').str.replace(',','')
    acc_clean.append(a)
acc_clean[0]=acc_clean[0].astype(int)
acc_clean[1]=acc_clean[1].astype(int)
acc_clean[2]=acc_clean[2].astype(int)
acc_clean[3]=acc_clean[3].astype(int)
acc_clean[4]=acc_clean[4].astype(int)
acc_clean[5]=acc_clean[5].astype(int)
acc_clean[6]=acc_clean[6].astype(int)
acc_clean[7]=acc_clean[7].astype(int)
acc_clean[8]=acc_clean[8].astype(int)
acc_clean[9]=acc_clean[9].astype(int)
acc_clean[10]=acc_clean[10].astype(int)
acc_clean[11]=acc_clean[11].astype(int)




In [31]:
all_data = p_prices.join(acc_clean[0])


In [32]:
fechas=all_data.columns.get_values()
fechas=fechas[0:-1]

In [44]:
fechas2=pd.to_datetime(fechas)

In [33]:
precios=all_data.iloc[:,0:-1].transpose()
cantidades=all_data['Acciones']

In [34]:
totales=np.dot(precios,cantidades)
totales=pd.DataFrame(totales,index=fechas)

In [36]:
navs_diarios1=totales/nassets[0]

In [37]:
rendimiento_final1=navs_diarios1.iloc[-1]/navs_diarios1.iloc[0]-1

In [38]:
navs_diarios1=pd.DataFrame(navs_diarios1)

In [39]:
rends_diarios1=((navs_diarios1-navs_diarios1.shift(1))/navs_diarios1.shift(1)).iloc[1:]

In [40]:
desv_diaria1=np.std(rends_diarios1)

In [41]:
rends=[]
for i in range(len(df_fechas)-1):
    rend=rend1=navs_diarios1.transpose()[df_fechas[i+1]]/navs_diarios1.transpose()[df_fechas[i]]-1
    rends.append(rend)
rend1=pd.DataFrame(rends,index=df_fechas[1:])



In [42]:
sharpe1=(rend1-.02)/desv_diaria1

# Escenario 2 

In [45]:
p_prices2=df_prices[1].pivot('ticker','date','adj_close') #Genera la tabla pivote 
p_prices2=p_prices2.fillna(method='ffill',axis=1) #Se rellenan los NaN con el dato anterior disponible

In [48]:
all_data2=p_prices2.join(acc_clean[1])

In [49]:
precios2=all_data2.iloc[:,0:-1].transpose()
cantidades2=all_data2['Acciones']


In [50]:
totales2=np.dot(precios2,cantidades2)
totales2=pd.DataFrame(totales2, index= fechas2[(fechas2 >= df_fechas[1]) & (fechas2 <= df_fechas[12])] )

In [51]:
navs_diarios2=totales2/nassets[1]

In [52]:
rendimiento_final2=navs_diarios2.iloc[-1]/navs_diarios2.iloc[0]-1
navs_diarios2=pd.DataFrame(navs_diarios2)

In [53]:
rends2=[]
for i in range(len(df_fechas)-2):
    rend2=rendd=navs_diarios2.transpose()[df_fechas[i+2]]/navs_diarios2.transpose()[df_fechas[i+1]]-1
    rends2.append(rend2)
rends2=pd.DataFrame(rends2,index = df_fechas[2:])



In [54]:
a= rend1.loc[[(df_fechas[1])]]
rend2=a.append(rends2)

In [55]:
navs_diariosb=pd.DataFrame(navs_diarios1, index= fechas2[(fechas2 < df_fechas[1])] )
navs_diarios2=navs_diariosb.append(navs_diarios2)

In [56]:
rends_diarios2=((navs_diarios2-navs_diarios2.shift(1))/navs_diarios2.shift(1)).iloc[1:]
desv_diaria2=np.std(rends_diarios2)

In [57]:
sharpe2=(rend2-.02)/desv_diaria2

# Escenario 3 

In [58]:
p_prices3=df_prices[2].pivot('ticker','date','adj_close') #Genera la tabla pivote 
p_prices3=p_prices3.fillna(method='ffill',axis=1) #Se rellenan los NaN con el dato anterior disponible

In [59]:
all_data3=p_prices3.join(acc_clean[2])

In [60]:
precios3=all_data3.iloc[:,0:-1].transpose()
cantidades3=all_data3['Acciones']

In [61]:
totales3=np.dot(precios3,cantidades3)
totales3=pd.DataFrame(totales3, index= fechas2[(fechas2 >= df_fechas[2]) & (fechas2 <= df_fechas[12])] )

In [62]:
navs_diarios3=totales3/nassets[2]

In [63]:
rendimiento_final3=navs_diarios3.iloc[-1]/navs_diarios3.iloc[0]-1
navs_diarios3=pd.DataFrame(navs_diarios3)
rends_diarios3=((navs_diarios3-navs_diarios3.shift(1))/navs_diarios3.shift(1)).iloc[1:]
desv_diaria3=np.std(rends_diarios3)

In [64]:
rends3=[]
for i in range(len(df_fechas)-3):
    rend3=rendd=navs_diarios3.transpose()[df_fechas[i+3]]/navs_diarios3.transpose()[df_fechas[i+2]]-1
    rends3.append(rend3)
rends3=pd.DataFrame(rends3,index = df_fechas[3:])

In [65]:
b= rend2.loc[[(df_fechas[1] ),(df_fechas[2])]]
rend3=b.append(rends3)

In [66]:
navs_diariosb=pd.DataFrame(navs_diarios2, index= fechas2[(fechas2 < df_fechas[2])] )
navs_diarios3=navs_diariosb.append(navs_diarios3)
rends_diarios3=((navs_diarios3-navs_diarios3.shift(1))/navs_diarios3.shift(1)).iloc[1:]
desv_diaria3=np.std(rends_diarios3)

In [67]:
sharpe3=(rend3-.02)/desv_diaria3

# Escenario 4 

In [68]:
p_prices4=df_prices[3].pivot('ticker','date','adj_close') #Genera la tabla pivote 
p_prices4=p_prices4.fillna(method='ffill',axis=1) #Se rellenan los NaN con el dato anterior disponible

In [69]:
all_data4=p_prices4.join(acc_clean[3])

In [70]:
precios4=all_data4.iloc[:,0:-1].transpose()
cantidades4=all_data4['Acciones']

In [71]:
totales4=np.dot(precios4,cantidades4)
totales4=pd.DataFrame(totales4, index= fechas2[(fechas2 >= df_fechas[3]) & (fechas2 <= df_fechas[12])] )

In [72]:
navs_diarios4=totales4/nassets[3]

In [73]:
rendimiento_final4=navs_diarios4.iloc[-1]/navs_diarios4.iloc[0]-1
navs_diarios4=pd.DataFrame(navs_diarios4)
rends_diarios4=((navs_diarios4-navs_diarios4.shift(1))/navs_diarios4.shift(1)).iloc[1:]
desv_diaria4=np.std(rends_diarios4)

In [74]:
rends4=[]
for i in range(len(df_fechas)-4):
    rend4=rendd=navs_diarios4.transpose()[df_fechas[i+4]]/navs_diarios4.transpose()[df_fechas[i+3]]-1
    rends4.append(rend4)
rends4=pd.DataFrame(rends4,index = df_fechas[4:])

In [75]:
c= rend3.loc[[(df_fechas[1] ),(df_fechas[2]),(df_fechas[3])]]
rend4= c.append(rends4)

In [76]:
navs_diariosb=pd.DataFrame(navs_diarios3, index= fechas2[(fechas2 < df_fechas[3])] )
navs_diarios4=navs_diariosb.append(navs_diarios4)
rends_diarios4=((navs_diarios4-navs_diarios4.shift(1))/navs_diarios4.shift(1)).iloc[1:]
desv_diaria4=np.std(rends_diarios4)

In [77]:
sharpe4=(rend4-.02)/desv_diaria4

# Escenario 5 

In [78]:
p_prices5=df_prices[4].pivot('ticker','date','adj_close') #Genera la tabla pivote 
p_prices5=p_prices5.fillna(method='ffill',axis=1) #Se rellenan los NaN con el dato anterior disponible

In [79]:
all_data5=p_prices5.join(acc_clean[4])

In [80]:
precios5=all_data5.iloc[:,0:-1].transpose()
cantidades5=all_data5['Acciones']

In [81]:
totales5=np.dot(precios5,cantidades5)
totales5=pd.DataFrame(totales5, index= fechas2[(fechas2 >= df_fechas[4]) & (fechas2 <= df_fechas[12])] )

In [82]:
navs_diarios5=totales5/nassets[4]

In [83]:
rendimiento_final5=navs_diarios5.iloc[-1]/navs_diarios5.iloc[0]-1
navs_diarios5=pd.DataFrame(navs_diarios5)
rends_diarios5=((navs_diarios5-navs_diarios5.shift(1))/navs_diarios5.shift(1)).iloc[1:]
desv_diaria5=np.std(rends_diarios5)

In [84]:
rends5=[]
for i in range(len(df_fechas)-5):
    rend5=rendd=navs_diarios5.transpose()[df_fechas[i+5]]/navs_diarios5.transpose()[df_fechas[i+4]]-1
    rends5.append(rend5)
rends5=pd.DataFrame(rends5,index = df_fechas[5:])

In [85]:
d= rend4.loc[[(df_fechas[1] ),(df_fechas[2]),(df_fechas[3]),(df_fechas[4])]]
rend5= d.append(rends5)

In [86]:
navs_diariosb=pd.DataFrame(navs_diarios4, index= fechas2[(fechas2 < df_fechas[4])] )
navs_diarios5=navs_diariosb.append(navs_diarios5)
rends_diarios5=((navs_diarios5-navs_diarios5.shift(1))/navs_diarios5.shift(1)).iloc[1:]
desv_diaria5=np.std(rends_diarios5)

In [87]:
sharpe5=(rend5-.02)/desv_diaria5

# Escenario 6

In [88]:
p_prices6=df_prices[5].pivot('ticker','date','adj_close') #Genera la tabla pivote 
#p_prices=p_prices.fillna(p_prices.mean(axis=1))
p_prices6=p_prices6.fillna(method='ffill',axis=1) #Se rellenan los NaN con el dato anterior disponible

In [89]:
all_data6=p_prices6.join(acc_clean[5])

In [90]:
precios6=all_data6.iloc[:,0:-1].transpose()
cantidades6=all_data6['Acciones']

In [91]:
totales6=np.dot(precios6,cantidades6)
totales6=pd.DataFrame(totales6, index= fechas2[(fechas2 >= df_fechas[5]) & (fechas2 <= df_fechas[12])] )

In [92]:
navs_diarios6=totales6/nassets[5]

In [93]:
rendimiento_final6=navs_diarios6.iloc[-1]/navs_diarios6.iloc[0]-1
navs_diarios6=pd.DataFrame(navs_diarios6)
rends_diarios6=((navs_diarios6-navs_diarios6.shift(1))/navs_diarios6.shift(1)).iloc[1:]
desv_diaria6=np.std(rends_diarios6)

In [94]:
rends6=[]
for i in range(len(df_fechas)-6):
    rend6=rendd=navs_diarios6.transpose()[df_fechas[i+6]]/navs_diarios6.transpose()[df_fechas[i+5]]-1
    rends6.append(rend6)
rends6=pd.DataFrame(rends6,index = df_fechas[6:])


In [95]:
e= rend5.loc[[(df_fechas[1] ),(df_fechas[2]),(df_fechas[3]),(df_fechas[4]),(df_fechas[5])]]
rend6= e.append(rends6)

In [96]:
navs_diariosb=pd.DataFrame(navs_diarios5, index= fechas2[(fechas2 < df_fechas[5])] )
navs_diarios6=navs_diariosb.append(navs_diarios6)
rends_diarios6=((navs_diarios6-navs_diarios6.shift(1))/navs_diarios6.shift(1)).iloc[1:]
desv_diaria6=np.std(rends_diarios6)

In [97]:
sharpe6=(rend6-.02)/desv_diaria6

# Escenario 7 

In [98]:
p_prices7=df_prices[6].pivot('ticker','date','adj_close') #Genera la tabla pivote 
p_prices7=p_prices7.fillna(method='ffill',axis=1) #Se rellenan los NaN con el dato anterior disponible

In [99]:
all_data7=p_prices7.join(acc_clean[6])

In [100]:
precios7=all_data7.iloc[:,0:-1].transpose()
cantidades7=all_data7['Acciones']

In [101]:
totales7=np.dot(precios7,cantidades7)
totales7=pd.DataFrame(totales7, index= fechas2[(fechas2 >= df_fechas[6]) & (fechas2 <= df_fechas[12])] )

In [102]:
navs_diarios7=totales7/nassets[6]

In [103]:
rendimiento_final7=navs_diarios7.iloc[-1]/navs_diarios7.iloc[0]-1
navs_diarios7=pd.DataFrame(navs_diarios7)
rends_diarios7=((navs_diarios7-navs_diarios7.shift(1))/navs_diarios7.shift(1)).iloc[1:]
desv_diaria7=np.std(rends_diarios7)

In [104]:
rends7=[]
for i in range(len(df_fechas)-7):
    rend7=rendd=navs_diarios7.transpose()[df_fechas[i+7]]/navs_diarios7.transpose()[df_fechas[i+6]]-1
    rends7.append(rend7)
rends7=pd.DataFrame(rends7,index = df_fechas[7:])

In [105]:
f= rend6.loc[[(df_fechas[1] ),(df_fechas[2]),(df_fechas[3]),(df_fechas[4]),(df_fechas[5]),(df_fechas[6])]]
rend7= f.append(rends7)

In [106]:
navs_diariosb=pd.DataFrame(navs_diarios6, index= fechas2[(fechas2 < df_fechas[6])] )
navs_diarios7=navs_diariosb.append(navs_diarios7)
rends_diarios7=((navs_diarios7-navs_diarios7.shift(1))/navs_diarios7.shift(1)).iloc[1:]
desv_diaria7=np.std(rends_diarios7)

In [107]:
sharpe7=(rend7-.02)/desv_diaria7

# Escenario 8 

In [108]:
p_prices8=df_prices[7].pivot('ticker','date','adj_close') #Genera la tabla pivote 
p_prices8=p_prices8.fillna(method='ffill',axis=1) #Se rellenan los NaN con el dato anterior disponible

In [109]:
all_data8=p_prices8.join(acc_clean[7])

In [110]:
precios8=all_data8.iloc[:,0:-1].transpose()
cantidades8=all_data8['Acciones']

In [111]:
totales8=np.dot(precios8,cantidades8)
totales8=pd.DataFrame(totales8, index= fechas2[(fechas2 >= df_fechas[7]) & (fechas2 <= df_fechas[12])] )

In [112]:
navs_diarios8=totales8/nassets[7]

In [113]:
rendimiento_final8=navs_diarios8.iloc[-1]/navs_diarios8.iloc[0]-1
navs_diarios8=pd.DataFrame(navs_diarios8)
rends_diarios8=((navs_diarios8-navs_diarios8.shift(1))/navs_diarios8.shift(1)).iloc[1:]
desv_diaria8=np.std(rends_diarios8)

In [114]:
rends8=[]
for i in range(len(df_fechas)-8):
    rend8=rendd=navs_diarios8.transpose()[df_fechas[i+8]]/navs_diarios8.transpose()[df_fechas[i+7]]-1
    rends8.append(rend8)
rends8=pd.DataFrame(rends8,index = df_fechas[8:])

In [115]:
g= rend7.loc[[(df_fechas[1] ),(df_fechas[2]),(df_fechas[3]),(df_fechas[4]),(df_fechas[5]),(df_fechas[6]),(df_fechas[7])]]
rend8= g.append(rends8)

In [116]:
navs_diariosb=pd.DataFrame(navs_diarios7, index= fechas2[(fechas2 < df_fechas[7])] )
navs_diarios8=navs_diariosb.append(navs_diarios8)
rends_diarios8=((navs_diarios8-navs_diarios8.shift(1))/navs_diarios8.shift(1)).iloc[1:]
desv_diaria8=np.std(rends_diarios8)

In [117]:
sharpe8=(rend8-.02)/desv_diaria8